In [6]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd
import numpy as np
from scipy.special import gamma
from scipy import signal
import time
import itertools
from itertools import combinations
try:
    from tqdm import tqdm_notebooks as tqdm
except ImportError:
    tqdm = lambda x: x
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
###
from collections import defaultdict

from scipy.stats import pearsonr, zscore
from MFDFA import fgn
# Plot settings
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sns.set()
from tqdm import tqdm


import matplotlib as mpl
from matplotlib import cm

import sys
sys.path.insert(0, '/directory/tothe/handshakefile/')

from shogun import *
from scipy.stats import norm, laplace
import random
sys.path.append('/home/ak/Research/PaperCode/stylised_facts/hypothesisTesting')
sys.path.append('/home/ak/Research/PaperCode/stylised_facts/stylised_facts_data_utilities')
import shogunMMDutils as mmdutils

# import stylised_facts_data_utilities.createLOB as createLOB
# import gpyNARX as gpyNARX
#import stylised_facts_data_utilities.longtaildistr as longtail
%matplotlib inline

# plt.style.use(os.path.join(mpl.get_configdir(),'latexstyle.mplstyle'))

In [7]:
graphicsLocation = '/home/ak/Documents/Research/Papers/StylisedFactsPaper/figures'

In [8]:
def shogun_features(x, y):
    # create shogun features
    return [RealFeatures(x.reshape(1, len(x))), RealFeatures(y.reshape(1, len(y)))]

In [9]:
# removes extremities based on two inputs
def series_between_percentiles(series, upperValue,lowerValue):
    # Find out percentiles and get values in between
    lThres = np.percentile(series , lowerValue) # lower percentile
    uThres = np.percentile(series , upperValue) # higher percentile
    return series[(series > lThres) & (series <uThres)]

In [10]:
# import numerical libraries

from scipy import stats
# import tableone
try:
    from tableone import TableOne
except (ModuleNotFoundError, ImportError):
    # install on Colab
    !pip install tableone
    from tableone import TableOne

### Assumptions and Investigations

1/ By moving to the tick domain, the need to force each trade into a time slot is removed as one does
not need to force the trades into predetermined sampling points as in calendar time. Additionally, when using calendar time sampling

2/ can we recover normality in each of the assets and periods?

3/duration between trades is
also added to the subordination framework to account for
the speed with which market participants act in physical
time.

4/we bring qualitative empirical evidence that the impact of a single
trade depends on the intertrade time lags. We find that when the trading rate be- comes faster, the return variance per trade strongly increases and that this behavior persists at coarser time scales. 

5/So we answer the following question: is the realized variance created by 10 trades arriving over 10 seconds similar to the realized variance created by those very same trades had they arrived during 10 minutes? Any model that uses a transaction time clock implies that the two situations are similar. Our empirical findings show that they are not, and that trades arriving in a shorter duration have higher variance, thus showing the importance of the physical inter-trade time duration. 

In [ ]:
ActivityCLockData= '/media/ak/My Passport/Experiment Data/ActivityClockData/'
OHLCData= '/media/ak/My Passport/Experiment Data/OHLCData/'
folderList = os.listdir(ActivityCLockData)
OHLCDataList = os.listdir(OHLCData)
ActivityClockDataList = os.listdir(ActivityCLockData)

symbols =['FB1','JB1','FV1','G_1','DU1']

In [ ]:
JB1ClocksData = np.sort([s for s in os.listdir(ActivityCLockData) if 'Clocks' in s and ('JB1') in s])
FV1ClocksData = np.sort([s for s in os.listdir(ActivityCLockData) if 'Clocks' in s and ('FV1') in s])
G1ClocksData = np.sort([s for s in os.listdir(ActivityCLockData) if 'Clocks' in s and ('G_1') in s])

In [ ]:
DU1ClocksData = np.sort([s for s in os.listdir(ActivityCLockData) if 'Clocks' in s and ('DU1') in s])
FB1ClocksData = np.sort([s for s in os.listdir(ActivityCLockData) if 'Clocks' in s and ('FB1') in s])
AllClocksData = [DU1ClocksData, FB1ClocksData, JB1ClocksData, FV1ClocksData, G1ClocksData  ]

In [ ]:
pklidx = 0
pklClockFile = "".join((ActivityCLockData,DU1ClocksData[pklidx]))
pickleBarDF = pickle.load(open(pklClockFile , "rb"))

In [ ]:

types_of_bars = ['OriginalDF', 'TickBarDf', 'VolumeBarDf', 'DollarVolumeBarDf']
range_list = [x for x in range(0,10)]#list(range(0, 25))


In [ ]:
pklClockFile = ["".join((ActivityCLockData,clocksDataList[pklIdx])) for clocksDataList in AllClocksData for pklIdx in range(0,25)]

In [ ]:

x =clocksDataList[2].split("_")[4]
y = clocksDataList[2].split("_")[1]

In [ ]:
clock_dict = defaultdict(dict)
for clocksDataList in AllClocksData :
    for pklIdx in range(0,25): 
        clock_dict[clocksDataList[pklIdx].split("_")[4]][clocksDataList[pklIdx].split("_")[1]]  = "".join((ActivityCLockData,clocksDataList[pklIdx]))

In [ ]:
clocksDataList[0].split("_")[4]

### now lets get all the dataframes out

In [ ]:
tickBar_dfs = [pickle.load(open(pklClockFile , "rb"))['TickBarDf'] for pklClockFile in pklClockFile_locations]

In [ ]:
dataFrames = [pickle.load(open(pklClockFile , "rb"))[frame] for pklClockFile in pklClockFile_locations for frame in types_of_bars ]

## 1 - read from AllClocksData
## 2 - read from pklidx
## 3 - get pklClockFile
## 4 - load picklefile, and extract the dataframe
## 5  - take the 

## pick DU1 First ## 

In [ ]:
microPriceChanges = [dataFrames[i].MicroPrice.pct_change().dropna() for i in range(0, len(dataFrames))]

In [ ]:
test= microPriceChanges[0].dropna()

In [ ]:
len(series_between_percentiles(test, 0.9, 0.1))

### do a dict comprehension to create a dictionary with all the date you want
    

In [ ]:
mpc_dict = {i:series_between_percentiles(microPriceChanges[i].dropna(), 0.9, 0.1) for i in range(0,100)}

In [ ]:
sample_p = signal.resample(mpc_dict[0], 1000)
sample_q = signal.resample(mpc_dict[2], 1000)

In [ ]:
feat_p, feat_q = shogun_features(sample_p,sample_q)
mmdInstance = mmdutils.SignificanceResultsMMD(sample_p,sample_q)

In [ ]:
feat_p_2, feat_q_2 = mmdInstance.shogun_features(sample_p,sample_q)
# this function needs re-writing?

## unit-testing my functionality next

In [ ]:
mmd = QuadraticTimeMMD(feat_p_2, feat_q_2)

mmd_2 = mmdInstance.mmd()

In [ ]:
mmd.set_statistic_type(ST_BIASED_FULL)
kernel = GaussianKernel(10, 1)
mmd.set_kernel(kernel)
mmd.compute_statistic()

In [ ]:
kernel = GaussianKernel(10, 1)
mmd.set_kernel(kernel)
kernel1 = GaussianKernel(10, 0.1)
kernel2 = GaussianKernel(10, 1)
kernel3 = GaussianKernel(10, 10)
kernel4 = GaussianKernel(5, 0.15)
mmd.add_kernel(kernel1)
mmd.add_kernel(kernel2)
mmd.add_kernel(kernel3)
mmd.add_kernel(kernel4)

mmd.set_train_test_mode(True)
mmd.set_train_test_ratio(1)

In [ ]:
kernel_2 = GaussianKernel(20, 1)
mmd_2.set_kernel(kernel)
kernel1 = GaussianKernel(10, 0.1)
kernel2 = GaussianKernel(10, 1)
kernel3 = GaussianKernel(10, 10)
kernel4 = GaussianKernel(5, 0.15)
mmd_2.add_kernel(kernel1)
mmd_2.add_kernel(kernel2)
mmd_2.add_kernel(kernel3)
mmd_2.add_kernel(kernel4)

mmd_2.set_train_test_mode(True)
mmd_2.set_train_test_ratio(1)

In [ ]:
num_runs = 1
num_folds = 3
alpha = 0.05
mmd_2.set_kernel_selection_strategy(KSM_CROSS_VALIDATION, num_runs, num_folds, alpha)
mmd_2.select_kernel()
learnt_kernel_single = GaussianKernel.obtain_from_generic(mmd_2.get_kernel())
width = learnt_kernel_single.get_width()
print('Print best kernel width is:', width)


_, p_value_biased = mmdInstance.compute_the_p_value_biased(mmd=mmd_2, kernel=learnt_kernel_single)
_, p_value_unbiased = mmdInstance.compute_the_p_value_unbiased(mmd=mmd_2, kernel=learnt_kernel_single)


# mmdutils.SignificanceResultsMMD.test_by_hand(mmd=mmd_2, p_value=p_value_unbiased, alpha=alpha)
# mmdutils.SignificanceResultsMMD.full_two_sample(mmd=mmd_2, alpha=alpha)



In [ ]:
num_runs = 200
p_values = np.zeros(num_runs)

last = time.time()
for i in range(num_runs):
    feats_p, feats_q = shogun_features(fb1sample,jb1sample)
    width=1
    k = GaussianKernel(10, width)

    mmd = QuadraticTimeMMD()
    mmd.set_p(feats_p)
    mmd.set_q(feats_q)
    mmd.set_kernel(k)

    mmd.set_num_null_samples(50)
    stat = mmd.compute_statistic() # would be good if compute_p_value() with no arguments computed the statistic itself
    p_values[i] = mmd.compute_p_value(stat)

# does this look more or less uniform (it has to be)?
plt.hist(p_values);

In [ ]:
fig, ax = plt.subplots(1, 1)
sns.distplot(p_values, bins =50,color = "blue")
plt.axvline(x=np.percentile(p_values, 50), c='red',linestyle = 'dotted')
legend = ["median $p-$ value"]
ax.legend(loc='best', frameon=False)
_=plt.title("Distribution of $p$-values")
_=plt.xlabel("$x$")
title= 'DistributionPvaluesJB1DU1.png'
plt.savefig(os.path.join(graphicsLocation,title))




In [ ]:
from scipy.spatial.distance import squareform, pdist, cdist

def sq_distances(X,Y=None):
    """
    If Y=None, then this computes the distance between X and itself
    """
    assert(X.ndim==2)

    # IMPLEMENT: compute pairwise distance matrix. Don't use explicit loops, but the above scipy functions
    # if X=Y, use more efficient pdist call which exploits symmetry
    if Y is None:
        sq_dists = squareform(pdist(X, 'sqeuclidean'))
    else:
        assert(Y.ndim==2)
        assert(X.shape[1]==Y.shape[1])
        sq_dists = cdist(X, Y, 'sqeuclidean')

    return sq_dists
def gauss_kernel(X, Y=None, sigma=1.0):
    """
    Computes the standard Gaussian kernel k(x,y)=exp(- ||x-y||**2 / (2 * sigma**2))

    X - 2d array, samples on left hand side
    Y - 2d array, samples on right hand side, can be None in which case they are replaced by X
    
    returns: kernel matrix
    """

    # IMPLEMENT: compute squared distances and kernel matrix
    sq_dists = sq_distances(X,Y)
    K = np.exp(-sq_dists / (2 * sigma**2))
    return K

# IMPLEMENT
def linear_kernel(X, Y):
    return np.dot(X, Y.T)
def quadratic_time_mmd(X,Y,kernel):
    assert X.ndim == Y.ndim == 2
    K_XX = kernel(X,X)
    K_XY = kernel(X,Y)
    K_YY = kernel(Y,Y)
       
    n = len(K_XX)
    m = len(K_YY)
    
    # IMPLEMENT: unbiased MMD statistic (could also use biased, doesn't matter if we use permutation tests)
    np.fill_diagonal(K_XX, 0)
    np.fill_diagonal(K_YY, 0)
    mmd = np.sum(K_XX) / (n*(n-1))  + np.sum(K_YY) / (m*(m-1))  - 2*np.sum(K_XY)/(n*m)
    return mmd
def two_sample_permutation_test(test_statistic, X, Y, num_permutations, prog_bar=True):
    assert X.ndim == Y.ndim
    
    statistics = np.zeros(num_permutations)
    
    range_ = range(num_permutations)
    for i in range_:
        # concatenate samples
        if X.ndim == 1:
            Z = np.hstack((X,Y))
        elif X.ndim == 2:
            Z = np.vstack((X,Y))
            
        # IMPLEMENT: permute samples and compute test statistic
        perm_inds = np.random.permutation(len(Z))
        Z = Z[perm_inds]
        X_ = Z[:len(X)]
        Y_ = Z[len(X):]
        my_test_statistic = test_statistic(X_, Y_)
        statistics[i] = my_test_statistic
    return statistics
def plot_permutation_samples(null_samples, statistic=None, title=None):
    #plt.hist(
    sns.distplot(null_samples, bins=50, color='grey')
    plt.axvline(x=np.percentile(null_samples, 2.5), c='b')
    legend = ["95% quantiles"]
    if statistic is not None:
        plt.axvline(x=statistic, c='r', linewidth=3, linestyle='--')
        legend += ["Actual test statistic"]
    plt.legend(legend, loc='best')
    plt.axvline(x=np.percentile(null_samples, 97.5), c='b')
    plt.xlabel("Test statistic value")
    plt.ylabel("Counts")
    if title is not None:
        plt.savefig(os.path.join(graphicsLocation,title))

In [ ]:
X, Y = (fb1sample,jb1sample)
my_kernel = lambda X,Y : gauss_kernel(X,Y,sigma=0.2)
my_mmd = lambda X,Y : quadratic_time_mmd(X[:,np.newaxis],Y[:,np.newaxis], my_kernel)
num_permutations = 200
statistics = two_sample_permutation_test(my_mmd, X, Y, num_permutations)
my_statistic = my_mmd(X,Y)

In [ ]:
grid = np.linspace(np.min(X), np.max(X))
bins=150
my_kernel = lambda X,Y : gauss_kernel(X,Y, sigma=0.5)
#sns.distplot(X, bins =bins, color = "skyblue", label ="FB1")
sns.distplot(Y, bins =bins,color = "red", label ="JB1")
# plt.title("Witness function")
# # IMPLEMENT: evaluate MMD witness function on grid
# phi_X = np.mean(my_kernel(X[:,np.newaxis], grid[:,np.newaxis]), axis=0)
# phi_Y = np.mean(my_kernel(Y[:,np.newaxis], grid[:,np.newaxis]), axis=0)
# witness = phi_X-phi_Y
# plt.xlabel("$x$")
# plt.ylabel("$p(x)$")
# _=plt.legend([ 'FB1','JB1'])
# plt.plot(grid, witness, linestyle ='--')